In [24]:
import time

In [ ]:
import requests
import time
from random import random
from bs4 import BeautifulSoup
import pandas as pd

data = []
counter = 0
control = True 
def tiki_scraper(category_url):
    page = 205
    while True:
        print(page)
        
        # generates random delay between 0.5 and 1.5 seconds:
        delay = random() + 0.5

        # defines target page url 
        current = category_url + str(page)

        # general url business and creating soup obj:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

        r = requests.get(current, headers=headers)
        soup = BeautifulSoup(r.text, "html.parser")

        # populates list of all products on target page:
        products = soup.find_all("a", {"class": "product-item"}) 

        while len(products) == 0:
            r = requests.get(current, headers=headers)
            soup = BeautifulSoup(r.text, "html.parser")
            products = soup.find_all("a", {"class": "product-item"})
            counter += 1
            if counter == 20:
                control = False 
                break

        
        if not control:
            break 
        print(f"Currently scraping {len(products)} products from page {page}.")
      
        for product in products:
            #creates dictionary of product info:
            d = {}
            d['product_id'] = (product['href'].split('-')[-1])[:-5]    
            d['title'] = product.find('div', {'class':'name'}).text
            d['price'] = product.find('div', {'class':'price-discount__price'}).text
            d['image_url'] = product.img['src']
            d['free_shipping'] = bool(product.find('div', {'class':'badge-top'}))
            d['reviewer_tot'] = (product.find('div', {'class':'review'}).text).strip('()')
            data.append(d)
        page += 1

        #delays next loop initiation
        time.sleep(delay)


In [ ]:
tiki_scraper('https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner&page=')


In [ ]:
table = pd.DataFrame(data)
table.head()